In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%capture
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

In [1]:
!pip install -U pyarrow "datasets==4.3.0"

In [3]:
import torch
from unsloth import FastLanguageModel

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Unsloth import OK ✅")

Torch: 2.6.0+cu124
CUDA available: True
Unsloth import OK ✅


In [4]:
from datasets import load_dataset
import re

ds = load_dataset("namnguyenba2003/QA-law-dataset", split="train")  # 60k rows :contentReference[oaicite:1]{index=1}

# --- 1) Keyword whitelist (match thì GIỮ) ---
TRAFFIC = r"""
giao\s*thông|đường\s*bộ|trật\s*tự\s*an\s*toàn\s*giao\s*thông|
xe\s*máy|mô\s*tô|ô\s*tô|xe\s*đạp|xe\s*tải|xe\s*khách|
GPLX|giấy\s*phép\s*lái\s*xe|bằng\s*lái|
vượt\s*đèn\s*đỏ|đi\s*sai\s*làn|lấn\s*làn|vượt\s*quá\s*tốc\s*độ|quá\s*tốc\s*độ|
nồng\s*độ\s*cồn|ma\s*túy|cồn|
mũ\s*bảo\s*hiểm|biển\s*báo|vạch\s*kẻ\s*đường|đèn\s*tín\s*hiệu|
Cảnh\s*sát\s*giao\s*thông|CSGT|
Nghị\s*định\s*100/2019|Nghị\s*định\s*123/2021|Nghị\s*định\s*168/2024|
trừ\s*điểm|phục\s*hồi\s*điểm
"""

CIVIL = r"""
bộ\s*luật\s*dân\s*sự|BLDS|
hợp\s*đồng|giao\s*kết|vi\s*phạm\s*hợp\s*đồng|hủy\s*hợp\s*đồng|đơn\s*phương|
đặt\s*cọc|phạt\s*vi\s*phạm|bồi\s*thường|thiệt\s*hại|trách\s*nhiệm\s*bồi\s*thường|
đòi\s*nợ|nghĩa\s*vụ\s*trả\s*nợ|lãi\s*suất|lãi\s*chậm\s*trả|
chuyển\s*nhượng|tặng\s*cho|thừa\s*kế|di\s*chúc|
quyền\s*sở\s*hữu|chiếm\s*hữu|tài\s*sản|mua\s*bán|vay|mượn|
tranh\s*chấp\s*dân\s*sự
"""

# --- 2) Blacklist (match thì LOẠI) để tránh kéo vào các mảng khác trong dataset ---
BLACKLIST = r"""
hàng\s*không|sân\s*bay|hải\s*quan|thuế|chứng\s*khoán|xây\s*dựng|đất\s*đai|
lao\s*động|bhxh|bảo\s*hiểm\s*xã\s*hội|hình\s*sự|tố\s*tụng\s*hình\s*sự|
doanh\s*nghiệp|đầu\s*tư|thương\s*mại|sở\s*hữu\s*trí\s*tuệ|y\s*tế
"""

traffic_pat = re.compile(TRAFFIC, re.I | re.X)
civil_pat   = re.compile(CIVIL,   re.I | re.X)
black_pat   = re.compile(BLACKLIST, re.I | re.X)

def keep(x):
    q = (x.get("question") or "")
    a = (x.get("answer") or "")
    u = (x.get("url") or "")
    text = f"{q}\n{a}\n{u}"

    # whitelist hit?
    hit = bool(traffic_pat.search(text) or civil_pat.search(text))
    if not hit:
        return False

    # blacklist: nếu dính quá rõ ngoài ngành thì loại
    if black_pat.search(text):
        # vẫn cho phép “ngoại lệ” nếu nó cũng dính rất mạnh giao thông/dân sự
        if not (traffic_pat.search(text) or civil_pat.search(text)):
            return False

    return True

filtered = ds.filter(keep)

print("Total:", len(ds), "Filtered:", len(filtered))

# (khuyên) dedup theo question để giảm trùng
seen = set()
def dedup(ex):
    q = (ex["question"] or "").strip().lower()
    if q in seen: 
        return False
    seen.add(q)
    return True

filtered = filtered.filter(dedup)
print("After dedup:", len(filtered))

# Lưu ra file để train nhanh
#filtered.to_json("qa_traffic_civil.jsonl", orient="records", lines=True, force_ascii=False)


README.md:   0%|          | 0.00/384 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60000 [00:00<?, ? examples/s]

Total: 60000 Filtered: 21260


Filter:   0%|          | 0/21260 [00:00<?, ? examples/s]

After dedup: 21248


In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
dtype = None
model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    dtype = dtype, # None for auto detection
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.12.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
messages = [
    {'role': 'system', 'content': 'Bạn là chuyên gia giàu kinh nghiệm trong lĩnh vực dân sự và rất am hiểu về luật giao thông', 'thinking': None},
    {"role": "user", "content": "“Dùng bằng lái xe ô tô giả bị phạt bao nhiêu 2025? "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 14 Dec 2025

Bạn là chuyên gia giàu kinh nghiệm trong lĩnh vực dân sự và rất am hiểu về luật giao thông<|eot_id|><|start_header_id|>user<|end_header_id|>

“Dùng bằng lái xe ô tô giả bị phạt bao nhiêu 2025?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Theo quy định của pháp luật Việt Nam, sử dụng bằng lái xe ô tô giả là hành vi vi phạm pháp luật về giao thông đường bộ.

Theo Điều 4 Nghị định 100/2019/NĐ-CP, người sử dụng bằng lái xe ô tô giả bị xử phạt từ 1 triệu đồng đến 2 triệu đồng.<|eot_id|>


In [8]:
filtered

Dataset({
    features: ['url', 'time', 'question', 'answer'],
    num_rows: 21248
})

In [9]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False,
        )
        for convo in convos
    ]
    return {"text": texts}


from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("namnguyenba2003/QA-law-dataset", split="train")
dataset

Dataset({
    features: ['url', 'time', 'question', 'answer'],
    num_rows: 60000
})

In [10]:
SYSTEM_PROMPT = (
    "Bạn là trợ lý pháp lý tiếng Việt, chuyên trả lời câu hỏi về pháp luật Việt Nam "
    "ngắn gọn, chính xác và dễ hiểu."
)

def to_messages(example):
    q = "" if example["question"] is None else str(example["question"])
    a = "" if example["answer"] is None else str(example["answer"])
    return {
        "messages": [
            {"role": "system",    "content": SYSTEM_PROMPT},
            {"role": "user",      "content": q},
            {"role": "assistant", "content": a},
        ]
    }

filtered = filtered.filter(
    lambda ex: ex["question"] is not None and ex["answer"] is not None
)

filtered = filtered.map(
    to_messages,
    remove_columns=["url", "time", "question", "answer"],
)
processed_dataset = filtered.map(formatting_prompts_func, batched=True)

print(processed_dataset)
print(processed_dataset[0]["text"][:500])  # xem thử 1 mẫu

Filter:   0%|          | 0/21248 [00:00<?, ? examples/s]

Map:   0%|          | 0/21248 [00:00<?, ? examples/s]

Map:   0%|          | 0/21248 [00:00<?, ? examples/s]

Dataset({
    features: ['messages', 'text'],
    num_rows: 21248
})
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 14 Dec 2025

Bạn là trợ lý pháp lý tiếng Việt, chuyên trả lời câu hỏi về pháp luật Việt Nam ngắn gọn, chính xác và dễ hiểu.<|eot_id|><|start_header_id|>user<|end_header_id|>

Nhiệm vụ, giải pháp thực hiện mục tiêu kế hoạch chuyển đổi số của Bộ Xây dựng năm 2025 là gì?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Nội dung này được quy định tại Mục 2 Kế hoạch ban hành kèm theoQ


In [11]:
from trl import SFTConfig, SFTTrainer
from transformers import EarlyStoppingCallback

early_stop = EarlyStoppingCallback(
    early_stopping_patience = 3,   # sau 3 lần eval loss không cải thiện thì dừng
    early_stopping_threshold = 0.0 # cải thiện phải < 0.0 mới tính là không tốt hơn
)

new_dataset = processed_dataset.train_test_split(
    test_size = 0.01,  # 1% làm dev/eval
    shuffle   = True,
    seed      = 3407,
)

train_dataset = new_dataset["train"]
eval_dataset  = new_dataset["test"]

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        
        eval_strategy = "steps", 
        eval_steps = 100,
        save_strategy = "steps",
        save_steps = 100,
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        
        
    ),
    callbacks = [early_stop],
)

[trl.trainer.sft_trainer|WARNING]You are using a per_device_train_batch_size of 1 with padding-free training. Using a batch size of 1 anihilate the benefits of padding-free training. Please consider increasing the batch size to at least 2.


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/21035 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/213 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [12]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21,035 | Num Epochs = 1 | Total steps = 5,259
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 12,156,928 of 3,224,906,752 (0.38% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,1.217000,1.151952
200,1.214300,1.104192
300,1.095000,1.077592
400,1.023600,1.048332
500,1.013000,1.030326
600,0.619600,1.012292
700,0.870200,0.995531
800,1.181900,0.978182
900,1.222800,0.968035
1000,0.872200,0.955171


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [13]:
messages = [
    {'role': 'system', 'content': 'Bạn là chuyên gia giàu kinh nghiệm trong lĩnh vực dân sự và rất am hiểu về luật giao thông', 'thinking': None},
    {"role": "user", "content": "“Dùng bằng lái xe ô tô giả bị phạt bao nhiêu 2025? "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 14 Dec 2025

Bạn là chuyên gia giàu kinh nghiệm trong lĩnh vực dân sự và rất am hiểu về luật giao thông<|eot_id|><|start_header_id|>user<|end_header_id|>

“Dùng bằng lái xe ô tô giả bị phạt bao nhiêu 2025?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Căn cứ theo điểm a khoản 5 Điều 28Nghị định 168/2024/NĐ-CPquy định như sau:
Điều 28. Xử phạt người điều khiển xe ô tô và các loại xe tương tự xe ô tô vi phạm quy định về điều kiện của phương tiện khi tham gia giao thông
[...]
5. Phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
a) Sử dụng Giấy phép lái xe không do cơ quan có thẩm quyền cấp (hoặc cấp không đúng thời hạn, hết hạn sử dụng), Giấy phép lái xe bị tẩy xóa, Giấy phép lái xe không còn hiệu lực, Giấy phép lái xe không hợp lệ hoặc sử dụng Giấy phép lái xe không hợp lệ;
[...]
10. Ngoài việc b

In [17]:
model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")

('finetuned_model/tokenizer_config.json',
 'finetuned_model/special_tokens_map.json',
 'finetuned_model/chat_template.jinja',
 'finetuned_model/tokenizer.json')

In [15]:
if False:
  from unsloth import FastLanguageModel
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = "finetuned_model", # YOUR MODEL YOU USED FOR TRAINING
      max_seq_length = 1024,
      dtype = None,
      load_in_4bit = True,
  )

messages = [
    {'role': 'system', 'content': 'Bạn là chuyên gia giàu kinh nghiệm trong lĩnh vực dân sự và rất am hiểu về luật giao thông', 'thinking': None},
    {"role": "user", "content": "“Dùng bằng lái xe ô tô giả bị phạt bao nhiêu 2025? "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 14 Dec 2025

Bạn là chuyên gia giàu kinh nghiệm trong lĩnh vực dân sự và rất am hiểu về luật giao thông<|eot_id|><|start_header_id|>user<|end_header_id|>

“Dùng bằng lái xe ô tô giả bị phạt bao nhiêu 2025?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Căn cứ khoản 3 Điều 15Nghị định 168/2024/NĐ-CPquy định như sau:
Điều 15. Xử phạt, trừ điểm giấy phép lái xe đối với chủ phương tiện, người điều khiển phương tiện vi phạm quy định về điều kiện của người điều khiển phương tiện khi tham gia giao thông
[...]
3. Phạt tiền từ 2.000.000 đồng đến 3.000.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Dùng bằng lái xe ô tô giả;
b) Dùng bằng lái xe ô tô không phù hợp với loại xe đang điều khiển;
c) Dùng bằng lái xe ô tô không có giấy phép lái xe hoặc sử dụng giấy phép lái xe không do cơ quan có thẩm quyền cấp, giấy phép lái xe đã hết hạn sử dụng, giấy phép lái xe bị tẩy

In [ ]:
if False:
  from unsloth import FastLanguageModel
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = "finetuned_model", # YOUR MODEL YOU USED FOR TRAINING
      max_seq_length = 1024,
      dtype = None,
      load_in_4bit = True,
  )

messages = [
    {'role': 'system', 'content': 'Bạn là chuyên gia giàu kinh nghiệm trong lĩnh vực dân sự và rất am hiểu về luật giao thông', 'thinking': None},
    {"role": "user", "content": "“Dùng bằng lái xe ô tô giả bị phạt bao nhiêu 2025? "},
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
    return_dict = True,
    reasoning_effort = "low",
).to(model.device)
from transformers import TextStreamer
_ = model.generate(**inputs, max_new_tokens = 512, streamer = TextStreamer(tokenizer))

### Tải model về

In [18]:
merged_dir = "/kaggle/working/finetuned_model"
model.save_pretrained_merged(merged_dir, tokenizer, save_method="merged_16bit")

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/2 [00:00<?, ?it/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [00:26<00:00, 13.26s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:40<00:00, 20.23s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/finetuned_model`
